$ docker pull tensorflow/serving

Pulls a tensorflow/serving image from docker hub.

$ docker run -d --name serving_base_container tensorflow/serving

Runs a tensorflow/serving container in the background(-d) named serving_base_container(or name of choice)

$ docker cp /home/carl/Dropbox/dev/bird_classifier/bird_classifier serving_base_container:/models/bird_classifier

Copies the directory containing the bird_classifier model into the container directory.

$ docker commit --change "ENV MODEL_NAME bird_classifier" serving_base_container bird_classifier

Changes the currently running serving_base_container container by setting the MODEL_NAME environment variable(which is used by tensorflow serving) equal to "bird_classifier" and renaming the new container "bird_classifier".

$ docker kill serving_base_container

Kill the currently running serving_base_container.

$ docker run -it -p 8500:8500 -p 8501:8501 --name bird_classifier bird_classifier




In [1]:
import json
import numpy as np
import requests

In [2]:
CLASS_NAMES = np.array(['ALBATROSS', 'ALEXANDRINE PARAKEET', 'AMERICAN AVOCET',
       'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN GOLDFINCH',
       'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART',
       'ANHINGA', 'ANNAS HUMMINGBIRD', 'ARARIPE MANAKIN', 'BALD EAGLE',
       'BALTIMORE ORIOLE', 'BANANAQUIT', 'BAR-TAILED GODWIT', 'BARN OWL',
       'BARN SWALLOW', 'BAY-BREASTED WARBLER', 'BELTED KINGFISHER',
       'BIRD OF PARADISE', 'BLACK FRANCOLIN', 'BLACK SKIMMER',
       'BLACK SWAN', 'BLACK THROATED WARBLER', 'BLACK VULTURE',
       'BLACK-CAPPED CHICKADEE', 'BLACK-NECKED GREBE',
       'BLACKBURNIAM WARBLER', 'BLUE GROUSE', 'BLUE HERON', 'BOBOLINK',
       'BROWN THRASHER', 'CACTUS WREN', 'CALIFORNIA CONDOR',
       'CALIFORNIA GULL', 'CALIFORNIA QUAIL', 'CAPE MAY WARBLER',
       'CARMINE BEE-EATER', 'CASPIAN TERN', 'CASSOWARY',
       'CHARA DE COLLAR', 'CHIPPING SPARROW', 'CINNAMON TEAL',
       'COCK OF THE  ROCK', 'COCKATOO', 'COMMON GRACKLE',
       'COMMON HOUSE MARTIN', 'COMMON LOON', 'COMMON POORWILL',
       'COMMON STARLING', 'COUCHS KINGBIRD', 'CRESTED AUKLET',
       'CRESTED CARACARA', 'CROW', 'CROWNED PIGEON', 'CUBAN TODY',
       'CURL CRESTED ARACURI', 'DARK EYED JUNCO', 'DOVEKIE',
       'DOWNY WOODPECKER', 'EASTERN BLUEBIRD', 'EASTERN MEADOWLARK',
       'EASTERN ROSELLA', 'EASTERN TOWEE', 'ELEGANT TROGON',
       'ELLIOTS  PHEASANT', 'EMPEROR PENGUIN', 'EMU', 'EURASIAN MAGPIE',
       'EVENING GROSBEAK', 'FLAME TANAGER', 'FLAMINGO', 'FRIGATE',
       'GILA WOODPECKER', 'GLOSSY IBIS', 'GOLD WING WARBLER',
       'GOLDEN CHLOROPHONIA', 'GOLDEN EAGLE', 'GOLDEN PHEASANT',
       'GOULDIAN FINCH', 'GRAY CATBIRD', 'GRAY PARTRIDGE', 'GREEN JAY',
       'GREY PLOVER', 'GUINEAFOWL', 'HAWAIIAN GOOSE', 'HOODED MERGANSER',
       'HOOPOES', 'HORNBILL', 'HOUSE FINCH', 'HOUSE SPARROW',
       'HYACINTH MACAW', 'INDIGO BUNTING', 'JABIRU', 'JAVAN MAGPIE',
       'KILLDEAR', 'KING VULTURE', 'LARK BUNTING', 'LILAC ROLLER',
       'LONG-EARED OWL', 'MALEO', 'MALLARD DUCK', 'MANDRIN DUCK',
       'MARABOU STORK', 'MASKED BOOBY', 'MIKADO  PHEASANT',
       'MOURNING DOVE', 'MYNA', 'NICOBAR PIGEON', 'NORTHERN CARDINAL',
       'NORTHERN FLICKER', 'NORTHERN GANNET', 'NORTHERN GOSHAWK',
       'NORTHERN JACANA', 'NORTHERN MOCKINGBIRD', 'NORTHERN RED BISHOP',
       'OCELLATED TURKEY', 'OSPREY', 'OSTRICH', 'PAINTED BUNTIG',
       'PARADISE TANAGER', 'PARUS MAJOR', 'PEACOCK', 'PELICAN',
       'PEREGRINE FALCON', 'PINK ROBIN', 'PUFFIN', 'PURPLE FINCH',
       'PURPLE GALLINULE', 'PURPLE MARTIN', 'PURPLE SWAMPHEN', 'QUETZAL',
       'RAINBOW LORIKEET', 'RED FACED CORMORANT', 'RED HEADED WOODPECKER',
       'RED THROATED BEE EATER', 'RED WINGED BLACKBIRD',
       'RED WISKERED BULBUL', 'RING-NECKED PHEASANT', 'ROADRUNNER',
       'ROBIN', 'ROCK DOVE', 'ROSY FACED LOVEBIRD', 'ROUGH LEG BUZZARD',
       'RUBY THROATED HUMMINGBIRD', 'RUFOUS KINGFISHER', 'RUFUOS MOTMOT',
       'SAND MARTIN', 'SCARLET IBIS', 'SCARLET MACAW', 'SHOEBILL',
       'SNOWY EGRET', 'SORA', 'SPLENDID WREN', 'SPOONBILL',
       'STORK BILLED KINGFISHER', 'STRAWBERRY FINCH', 'TAIWAN MAGPIE',
       'TEAL DUCK', 'TIT MOUSE', 'TOUCHAN', 'TRUMPTER SWAN',
       'TURKEY VULTURE', 'TURQUOISE MOTMOT', 'VARIED THRUSH',
       'VENEZUELIAN TROUPIAL', 'VERMILION FLYCATHER',
       'VIOLET GREEN SWALLOW', 'WESTERN MEADOWLARK',
       'WHITE CHEEKED TURACO', 'WILD TURKEY', 'WILSONS BIRD OF PARADISE',
       'WOOD DUCK', 'YELLOW HEADED BLACKBIRD'], dtype='<U25')


def get_top_n(predictions,n,class_labels):
  num_preds = predictions.shape[0]
  pred_list = []
  for pred in range(num_preds):
    topn_dict={}
    topn_indices = np.argpartition(predictions[pred], -n)[-n:]
    for each in range(topn_indices.shape[0]):
      bird_name = class_labels[topn_indices[each]]
      y_prob = predictions[pred][topn_indices[each]]
      topn_dict.update({bird_name : y_prob})
    topn_dict_ordered = sorted(topn_dict.items(), key=lambda item: item[1] , reverse=True)
    pred_list.append(topn_dict_ordered)
  return pred_list


In [3]:
image = np.load('/home/carl/Dropbox/dev/Data/STORK_BILLED_KINGFISHER.npy')

In [4]:
input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": image.tolist(),
})

In [7]:
SERVER_URL = 'http://localhost:8501/v1/models/bird_classifier:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status()
response = response.json()

In [8]:
preds = np.array(response['predictions']).round(4)
get_top_n(preds, 5, CLASS_NAMES)

[[('STORK BILLED KINGFISHER', 0.997),
  ('RUFUOS MOTMOT', 0.0012),
  ('RUFOUS KINGFISHER', 0.0011),
  ('TURQUOISE MOTMOT', 0.0002),
  ('ANHINGA', 0.0001)]]